In [5]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

from sae.sparse_autoencoder import *
from sae.activation_store import *
from sae.train import ModelTrainer
from sae.config import create_config, Config
from sae.metrics import *
from sae.utils import get_blog_checkpoint, get_blog_sparsity, create_lineplot_histogram
from unlearning.metrics import calculate_MCQ_metrics, all_permutations, calculate_metrics_rmu

from transformer_lens import HookedTransformer, utils
from sae.metrics import compute_metrics_post_by_text

import plotly.express as px
import plotly.graph_objs as go
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
torch.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

hf_model_name = "google/gemma-2b-it"
transformer_lens_model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(hf_model_name)

# hf_model_name = 'eoinf/gemma_2b_it_rmu_s60_a1000'
# hf_model = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto') #.to("cuda")
# RMU_model = HookedTransformer.from_pretrained(transformer_lens_model_name, hf_model=hf_model, tokenizer=tokenizer)

In [3]:
# from unlearning.var import gemma_2b_it_rmu_model_names

# metrics_lst = []



        
# for rmu_model_name in gemma_2b_it_rmu_model_names:
#     hf_model = AutoModelForCausalLM.from_pretrained(rmu_model_name, torch_dtype='auto') #.to("cuda")
#     RMU_model = HookedTransformer.from_pretrained(transformer_lens_model_name, hf_model=hf_model, tokenizer=tokenizer)
    
#     ablated_metrics = calculate_metrics_rmu(model, dataset_names, compute_loss=True)
#     metrics_list.append(ablated_metrics)
#     print()
    

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
metrics_lst

100%|██████████| 688/688 [02:38<00:00,  4.33it/s]


In [7]:
np.where(metrics['is_correct'].reshape(-1, 24).sum(axis=1) == 24)

(array([  1,   2,   3,   4,   7,   9,  11,  12,  17,  18,  20,  21,  29,
         31,  32,  37,  39,  40,  49,  50,  52,  80, 162, 163]),)

In [8]:
metrics['is_correct'].reshape(-1, 24).sum(axis=1)

array([ 6., 24., 24., 24., 24., 22.,  6., 24.,  6., 24.,  6., 24., 24.,
        4., 12., 20.,  6., 24., 24., 10., 24., 24., 14.,  6.,  5.,  6.,
        6., 10.,  6., 24.,  6., 24., 24.,  6.,  6.,  2.,  6., 24., 20.,
       24., 24.,  6.,  6.,  7.,  5.,  6.,  6.,  6.,  6., 24., 24., 12.,
       24.,  6.,  6.,  6.,  7.,  6.,  6., 23., 19.,  6.,  7.,  6.,  5.,
        6., 12.,  4.,  5., 20.,  9.,  3.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  2., 24., 13.,  4.,  6.,  6.,  6.,  6.,  6.,  6.,  6., 13.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  7.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6., 10.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  7.,  6.,  7.,  6.,  6.,  6.,
        6.,  6.,  6.,  8.,  6.,  6.,  6.,  6.,  6.,  6., 10.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  2.,  6., 10.,  6.,  6.,  6.,  6.,
        5.,  6.,  6.,  6.,  6.,  6., 24., 24., 23., 11.,  6.,  7.,  1.,
        6.,  6.,  9.], dtype=float32)

In [9]:
len([  1,   2,   3,   4,   7,   9,  11,  12,  17,  18,  20,  21,  29,
         31,  32,  37,  39,  40,  49,  50,  52,  80, 162, 163])

24

In [10]:
metrics

{'mean_correct': 0.3919573724269867,
 'total_correct': 1618,
 'is_correct': array([0., 1., 0., ..., 0., 0., 0.], dtype=float32),
 'output_probs': array([[2.50303417e-01, 8.79571028e-03, 5.84370971e-01, 1.46040231e-01],
        [2.65895516e-01, 5.83476573e-03, 6.20171905e-01, 9.64191407e-02],
        [2.65923947e-01, 1.07643753e-02, 5.67789912e-01, 1.44905016e-01],
        ...,
        [9.97600615e-01, 1.41504870e-06, 2.94048205e-05, 2.74580941e-07],
        [9.97656584e-01, 6.58353474e-06, 9.34167701e-07, 2.02913455e-07],
        [9.97717857e-01, 4.24272093e-06, 5.77771232e-07, 1.19147316e-07]],
       dtype=float32),
 'actual_answers': array([3, 2, 3, ..., 2, 1, 1]),
 'predicted_answers': array([2, 2, 2, ..., 0, 0, 0]),
 'predicted_probs': array([0.584371  , 0.6201719 , 0.5677899 , ..., 0.9976006 , 0.9976566 ,
        0.99771786], dtype=float32),
 'predicted_probs_of_correct_answers': array([1.4604023e-01, 6.2017190e-01, 1.4490502e-01, ..., 2.9404820e-05,
        6.5835347e-06, 4.2427